In [105]:
%load_ext autoreload
%autoreload 2


class Crit20010(Exception): #коннект к бд не найден. где именно не найден будет указано в сообщении на консоль
  def __init__(self,mes,kod):
    self.criterrno = kod
    self.criterrmess = mes
    super().__init__(f'Critical error {kod}: {mes}')  # правильный вызов родительского класса
    #print(f'критикал эксепшен. mess={self.__criterrmess}, kod={self.__criterrno}')
    #sys.exit(20010) #вобще выходим

class Crit20020(Exception): #ошибка в методе CONN в классе работы с бд
  def __init__(self,mes,kod):
    self.criterrno = kod
    self.criterrmess = mes
    super().__init__(f'Critical error {kod}: {mes}')  # правильный вызов родительского класса
    #print(f'критикал эксепшен. mess={self.__criterrmess}, kod={self.__criterrno}')
    #sys.exit(20010) #вобще выходим

from d_bd_cls import Bd_rabota
from d_krl_cls import Krauler



# все, что выше 19999 должно выкидывать из программы. аналогично полю level в t_log: записи со значениями выше 19999 означают критическую ошибку.

class Crit20000(Exception):
  # общий? эксепшен
  pass


import random
def worker():
  #коды выхода из Worker
  # 1 - на этапе коннекта к бд произошла обишка
  # 20000 - ошибка при попытке записать данные в лог на этапе выхода из Worker, основную работу должны были выполнить корректно.
  # 3 - не смог отключиться от СУБД в конце Worker
  #
  
  print("WORKER(): зашли в воркер")
  kod_vyhoda = 'Ok'
  regno = random.randrange(0, 9999)
  #print(f'WORKER(): присвоили ID={regno}')
  try:
    wsubd = Bd_rabota(regno) #создаем экземпляр объекта работы с бд на уровне воркера, для того, чтобы иметь возможность писать в лог бд события самого воркера
    wsubd.conn()
    wsubd.add_log('Worker','begin',1,f'выполнили CONNECT')
  except Crit20010 as e:
    print('не смог победить бд, текст ошибки:',e)
    return 1 #если при работе с бд возникла ошибка, то выходим, нечего и работать
 
  kr = Krauler( 'https://bamboo.phpbb.com/allPlans.action', wsubd)
  #kr = Krauler(regno, 'http://www.facebook.com/phpBB')
  #kr = Krauler(regno, 'https://biblio1.ru/test_diser1.html')
  try:
    kr.start1() #запуск краулера
    wsubd.add_log('Worker','вышел из start1 штатно',100,'k')
    wsubd.add_log('Worker','end',250,f'еще буду пытаться сделать disconnect')
    wsubd.disc()
    return (f' {kod_vyhoda} N={regno}')
  except Crit20010 as e: #этот 20010 для add_log
    print('крит',e.criterrmess)
    return e.criterrno #выходим из воркера
  except Crit20000:
    print('такого не должно быть')
    return 20000 #выходим из воркера
  #except Exception as e:
   # print(f'общий эксепшен. если будут сюда ошибки, нужно писать отдельные обрабочики эксепшенов: {e}')
    #return 21 #выходим из воркера


worker()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
WORKER(): зашли в воркер
2025-08-19 07:24:36.495366| cls_bd.INIT(idO=2338): инициализация объекта БД_работа.
2025-08-19 07:24:36.496366| cls_bd.CONNECT(idO=2338): метод коннекта к бд начало.
  2025-08-19 07:24:36.621532| cls_bd.CONNECT(idO=2338): Успешно подключено к базе данных
2025-08-19 07:24:37.508866| cls_bd.INSERT(idO=2338): метод INSERT записи в таблицу. урл=https://bamboo.phpbb.com/allPlans.action, status=1
  2025-08-19 07:24:37.509865| cls_bd.INSERT(idO=2338): живой коннект обнаружен
  2025-08-19 07:24:37.509865| cls_bd.INSERT(idO=2338): после сборки insert=
        insert into t_url set url="https://bamboo.phpbb.com/allPlans.action", url_red="", url_id_iskl=0, kod_otveta=200, busy=2338, status=1, 
        limit_recurs=10, limit_href=40, limit_size=1048576, url_size=56408, err_txt="", kolwo_recurs=0
    2025-08-19 07:24:37.511864| cls_bd.INSERT(idO=2338): Ошибка в методе INS_URL(). insert с

' Ok N=2338'

In [ ]:
resp = requests.get('https://biblio1.ru/test_diser1.html')

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
resp.raise_for_status()

In [ ]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
ll=[]

In [ ]:
for link in soup.find_all('a', href=True):
    print ('сам линк=',link)
    print('href=',link['href'])
    ll.append(link['href'])
    print('===')

print('-------------------')
ll

In [49]:
import d_bd_cls
dir(d_bd_cls)


['Bd_rabota',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [71]:
#!type d_bd_cls.py
!dir
%pycat d_bd_cls.py

 ’®¬ ў гбва®©бвўҐ C ­Ґ Ё¬ҐҐв ¬ҐвЄЁ.
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : CA93-D15B

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Users\nggnjhfgj

23.05.2025  01:31    <DIR>          .
23.05.2025  01:31    <DIR>          ..
17.02.2024  00:42    <DIR>          .anaconda
05.02.2024  11:57    <DIR>          .cache
02.11.2024  17:23    <DIR>          .conda
17.02.2024  00:42                25 .condarc
17.02.2024  00:41    <DIR>          .continuum
16.12.2023  14:40    <DIR>          .dotnet
27.10.2024  23:39               136 .gitconfig
23.05.2025  00:20    <DIR>          .ipynb_checkpoints
17.02.2024  00:43    <DIR>          .ipython
04.03.2025  15:37    <DIR>          .jupyter
04.09.2024  06:19    <DIR>          .matplotlib
05.02.2024  11:20    <DIR>          .nuget
06.09.2024  22:58    <DIR>          .spyder-py3
16.12.2023  14:50    <DIR>          .templateengine
22.05.2025  23:53    <DIR>          .virtual_documents
02.06.2023  13:52    <DIR>          3D Objects
02.06.2023  16:10    <DIR>          ansel
02.06.2023  13:52    

import threading
import mysql.connector
from mysql.connector import Error, errorcode
import time
from datetime import datetime

class Crit20010(Exception):
    def __init__(self, mes, kod):
        self.criterrno = kod
        self.criterrmess = mes

class Bd_rabota():
  def __init__(self, reg):
    print(f"{datetime.now()}| cls_bd.INIT(idO={reg}): инициализация объекта БД_работа.")
    self.__host='localhost'         # например, 'localhost' или IP адрес сервера
    self.__database='diser' # имя вашей базы данных
    self.__user='forpiton'     # имя пользователя
    self.__password='passwordpiton'   # пароль
    self.__connection = None #само соединение с СУБД. предполагается держать открытым
    self.__oregno = reg  #идентификатор для многопоточной работы
    #print(errorcode.ER_DUP_ENTRY, 'ddddddd')

  def conn(self): #коннект воркера к бд
    print(f"{datetime.now()}| cls_bd.CONNECT(idO={self.__oregno}): метод коннекта к бд начало.")
    if self.__connection:
      kod_log = self.__